In [19]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langgraph.graph.message import AnyMessage
from dotenv import load_dotenv
load_dotenv()
import os
from langchain_core.messages import HumanMessage
from langgraph.graph.message import add_messages, MessagesState
from typing import Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver


In [2]:
# os.environ['GOOGLE_API_KEY']

In [4]:
memory = MemorySaver()

In [7]:
class State(MessagesState):
    pass
    

In [8]:
graph = StateGraph(State)

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
)

In [10]:
def model_call(state):
    return {"messages" : [llm.invoke(state['messages'])]}

In [11]:
graph.set_entry_point('node1')
graph.add_node('node1', model_call)
graph.add_edge('node1', END)

In [12]:
graph = graph.compile()

In [13]:
type(graph)

langgraph.graph.state.CompiledStateGraph

In [14]:
messages = [HumanMessage('can you explain me how you get build')]

In [15]:
state = graph.invoke({'messages': messages})

In [16]:
new_message = [HumanMessage('ok now explain me how can i do it')]

In [17]:
state['messages'] + new_message

[HumanMessage(content='can you explain me how you get build', additional_kwargs={}, response_metadata={}, id='90e22518-79c9-4a65-b87e-4aa52f72b49d'),
 AIMessage(content='Okay, let\'s break down how I, as a large language model, am "built." It\'s a multi-faceted process involving data, architecture, training, and ongoing refinement.  Think of it like building a house: you need materials (data), a blueprint (architecture), construction workers (training algorithms), and ongoing maintenance (refinement).\n\n**1. Data: The Foundation**\n\n*   **Massive Text and Code Datasets:** I am trained on an enormous amount of text and code data. This data comes from a variety of sources, including:\n    *   **The Internet:**  Web pages, articles, blog posts, forums, and other publicly available text.\n    *   **Books:**  A vast collection of published books.\n    *   **Code Repositories:**  Code from platforms like GitHub, covering many programming languages.\n    *   **Other Sources:**  Scientific p

In [18]:
output = graph.invoke({'messages': state['messages'] + new_message})

In [31]:
print(output['messages'][-1].content)

Okay, let's break down how you can build your own language model. Keep in mind that building a state-of-the-art model like me requires significant resources (data, compute, expertise). However, you can definitely build smaller, simpler models to learn the fundamentals and experiment. Here's a roadmap:

**1. Define Your Goal and Scope:**

*   **What do you want your model to do?**  Be specific.  Examples:
    *   **Text generation:** Generate creative text, stories, poems, code, etc.
    *   **Text classification:** Classify text into categories (e.g., spam detection, sentiment analysis).
    *   **Question answering:** Answer questions based on a given context.
    *   **Translation:** Translate text from one language to another.
    *   **Summarization:** Summarize long documents.
*   **What is your target domain?**  Will it be general-purpose, or focused on a specific area (e.g., medical text, legal documents, financial news)?
*   **What resources do you have available?**  Consider y

### Memory for Multi-turn Conversation

In [23]:
memory_graph = StateGraph(State)

In [24]:
memory_graph.add_node('node1', model_call)
memory_graph.set_entry_point('node1')
memory_graph.set_finish_point('node1')

In [26]:
memory_graph = memory_graph.compile(checkpointer=memory)

In [28]:
config = {'configurable':{'thread_id':1}}

In [36]:
state = memory_graph.invoke({'messages': {'role':'user','content':'which is the most clean contry'}}, config)

In [40]:
messages

[HumanMessage(content='can you explain me how you get build', additional_kwargs={}, response_metadata={}, id='90e22518-79c9-4a65-b87e-4aa52f72b49d')]

In [41]:
for i in state['messages']:
    i.pretty_print()

================================ Human Message =================================

who are you
================================== Ai Message ==================================

I am a large language model, trained by Google.
================================ Human Message =================================

which is the most clean contry
================================== Ai Message ==================================

That's a really interesting question! It's tough to say definitively which country is "the most clean" because cleanliness can be measured in different ways. Here's a breakdown of factors and some countries often considered to be very clean:

**Factors to Consider:**

*   **Environmental Performance Index (EPI):** This index, produced by Yale and Columbia Universities, looks at environmental health and ecosystem vitality. It considers factors like air and water quality, sanitation, and pollution levels.
*   **Waste Management:** How well a country collects, processes, and di

In [ ]:
messages = [HumanMessage('w')]
memory_graph.invoke()

In [34]:
for i in memory.list(config):
    print(i)

In [ ]:
memory.get(con

In [48]:
graph.get_prompts(config)

[]

In [56]:
print(memory_graph.aget_state_history(config))

<async_generator object Pregel.aget_state_history at 0x10f113d30>


In [57]:
print(memory_graph.aget_state(config=config))

<coroutine object Pregel.aget_state at 0x10f5f2140>


/var/folders/81/8b_lw5jj52qfv4kmf14tk0p00000gn/T/ipykernel_36552/3467784115.py:1: RuntimeWarning: coroutine 'Pregel.aget_state' was never awaited
  print(memory_graph.aget_state(config=config))


In [59]:
memory_graph.get_state(config)

StateSnapshot(values={'messages': [HumanMessage(content='who are you', additional_kwargs={}, response_metadata={}, id='460aa9c5-96d5-4ab2-98ae-ff2b31c68326'), AIMessage(content='I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--aeb98977-bd8e-4f1d-ae4e-b50470fc9755-0', usage_metadata={'input_tokens': 3, 'output_tokens': 12, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='which is the most clean contry', additional_kwargs={}, response_metadata={}, id='6196b180-8f33-4f4d-8d8d-198024f135e4'), AIMessage(content='That\'s a really interesting question! It\'s tough to say definitively which country is "the most clean" because cleanliness can be measured in different ways. Here\'s a breakdown of factors and some countries often considered to be very clean:\n\n**Facto